In [3]:
import sys
import time
import datetime

In [4]:
TOPIC_Step2_NAME="Sahamyab-Tweets2"
KAFKA_SERVER="kafka-broker:29092"

In [5]:
import os
# https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html
# https://mvnrepository.com/artifact/org.apache.spark/spark-sql-kafka-0-10_2.12
# setup arguments
os.environ['PYSPARK_SUBMIT_ARGS']='--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2 pyspark-shell'

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *


spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("Step2_03-Hashtag-Processing") \
    .config("spark.executor.memory", "512mb") \
    .config("spark.executor.cores","1") \
    .config("spark.cores.max", "1") \
    .config("spark.sql.session.timeZone", "Asia/Tehran") \
    .getOrCreate()    
    

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9ddaca00-1f63-4550-bd03-c3cdc626df68;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.1.2/spark-sql-kafka-0-10_2.12-3.1.2.jar ...
	[SUCCESSFUL ] org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2!spark-sql-

In [6]:
spark.sparkContext.setLogLevel("ERROR")


In [7]:
# https://sparkbyexamples.com/spark/spark-sql-structtype-on-dataframe/
schema = StructType([StructField("id", StringType(), True),\
                         StructField("content", StringType(), True),\
                         StructField("sendTime", StringType(), True),\
                         StructField("sendTimePersian", StringType(), True),\
                         StructField("senderName", StringType(), True),\
                         StructField("senderUsername", StringType(), True),\
                         StructField("type", StringType(), True),\
                         StructField("hashtags", ArrayType(StringType()), True)
                    ])

In [8]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", KAFKA_SERVER) \
  .option("subscribe", TOPIC_Step2_NAME) \
  .option("startingOffsets", "earliest") \
  .load()

In [9]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [10]:

tweetsStringDF = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

tweetsDF = tweetsStringDF.select(from_json(col("value"), schema).alias("data")).select("data.*")
tweetsDF = tweetsDF.withColumn("timestamp", unix_timestamp("sendTime", "yyyy-MM-dd'T'HH:mm:ssz").cast('timestamp')) \
            .withColumn("persianYear", tweetsDF['sendTimePersian'].substr(0, 4)) \
            .withColumn("persianMonth", tweetsDF['sendTimePersian'].substr(6, 2)) \
            .withColumn("persianDay", tweetsDF['sendTimePersian'].substr(9, 2))


In [ ]:
tweetsDF.select(col("id"), col("sendTime"), col("senderName"), col("persianYear"),col("persianMonth"), col("persianDay"))\
      .writeStream \
      .format("console") \
      .outputMode("append") \
      .start() \
      .awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+---------+--------------------+----------+-----------+------------+----------+
|       id|            sendTime|senderName|persianYear|persianMonth|persianDay|
+---------+--------------------+----------+-----------+------------+----------+
|379568970|2021-10-08T17:42:37Z|  نیمه سوم|       1400|          07|        16|
|379568952|2021-10-08T17:42:25Z|       SHS|       1400|          07|        16|
|379568933|2021-10-08T17:42:16Z|      همای|       1400|          07|        16|
|379568893|2021-10-08T17:41:50Z|  Mohammad|       1400|          07|        16|
|379568850|2021-10-08T17:41:19Z|     ARASH|       1400|          07|        16|
|379568835|2021-10-08T17:41:15Z|   capitiz|       1400|          07|        16|
|379568776|2021-10-08T17:40:31Z|    مازیار|       1400|          07|        16|
|379568729|2021-10-08T17:39:53Z| صدای بورس|       1400|          07|        16|
|379568689|2021-10-08T1

-------------------------------------------
Batch: 2
-------------------------------------------
+---------+--------------------+----------+-----------+------------+----------+
|       id|            sendTime|senderName|persianYear|persianMonth|persianDay|
+---------+--------------------+----------+-----------+------------+----------+
|379630080|2021-10-08T18:09:54Z|      موفق|       1400|          07|        16|
+---------+--------------------+----------+-----------+------------+----------+

-------------------------------------------
Batch: 3
-------------------------------------------
+---------+--------------------+-------------+-----------+------------+----------+
|       id|            sendTime|   senderName|persianYear|persianMonth|persianDay|
+---------+--------------------+-------------+-----------+------------+----------+
|379629829|2021-10-08T18:09:53Z|         موفق|       1400|          07|        16|
|379627345|2021-10-08T18:09:15Z|          الی|       1400|          07|  

-------------------------------------------
Batch: 4
-------------------------------------------
+---------+--------------------+------------+-----------+------------+----------+
|       id|            sendTime|  senderName|persianYear|persianMonth|persianDay|
+---------+--------------------+------------+-----------+------------+----------+
|379630080|2021-10-08T18:09:54Z|        موفق|       1400|          07|        16|
|379629829|2021-10-08T18:09:53Z|        موفق|       1400|          07|        16|
|379627345|2021-10-08T18:09:15Z|         الی|       1400|          07|        16|
|379626436|2021-10-08T18:09:05Z|mobinvad9648|       1400|          07|        16|
+---------+--------------------+------------+-----------+------------+----------+

-------------------------------------------
Batch: 5
-------------------------------------------
+---------+--------------------+-------------+-----------+------------+----------+
|       id|            sendTime|   senderName|persianYear|persianM

-------------------------------------------
Batch: 6
-------------------------------------------
+---------+--------------------+----------+-----------+------------+----------+
|       id|            sendTime|senderName|persianYear|persianMonth|persianDay|
+---------+--------------------+----------+-----------+------------+----------+
|379630080|2021-10-08T18:09:54Z|      موفق|       1400|          07|        16|
|379629829|2021-10-08T18:09:53Z|      موفق|       1400|          07|        16|
+---------+--------------------+----------+-----------+------------+----------+

-------------------------------------------
Batch: 7
-------------------------------------------
+---------+--------------------+-------------+-----------+------------+----------+
|       id|            sendTime|   senderName|persianYear|persianMonth|persianDay|
+---------+--------------------+-------------+-----------+------------+----------+
|379627345|2021-10-08T18:09:15Z|          الی|       1400|          07|     

-------------------------------------------
Batch: 8
-------------------------------------------
+---------+--------------------+-------------+-----------+------------+----------+
|       id|            sendTime|   senderName|persianYear|persianMonth|persianDay|
+---------+--------------------+-------------+-----------+------------+----------+
|379630080|2021-10-08T18:09:54Z|         موفق|       1400|          07|        16|
|379629829|2021-10-08T18:09:53Z|         موفق|       1400|          07|        16|
|379627345|2021-10-08T18:09:15Z|          الی|       1400|          07|        16|
|379626436|2021-10-08T18:09:05Z| mobinvad9648|       1400|          07|        16|
|379626160|2021-10-08T18:09:00Z|      hossein|       1400|          07|        16|
|379622873|2021-10-08T18:08:20Z|    mirzaei.s|       1400|          07|        16|
|379621830|2021-10-08T18:08:06Z|مستر هاتسون 2|       1400|          07|        16|
|379621518|2021-10-08T18:08:03Z|     Mohammad|       1400|          07|  

...

-------------------------------------------
Batch: 100
-------------------------------------------
+---------+--------------------+----------+-----------+------------+----------+
|       id|            sendTime|senderName|persianYear|persianMonth|persianDay|
+---------+--------------------+----------+-----------+------------+----------+
|379630080|2021-10-08T18:09:54Z|      موفق|       1400|          07|        16|
+---------+--------------------+----------+-----------+------------+----------+

-------------------------------------------
Batch: 101
-------------------------------------------
+---------+--------------------+-------------+-----------+------------+----------+
|       id|            sendTime|   senderName|persianYear|persianMonth|persianDay|
+---------+--------------------+-------------+-----------+------------+----------+
|379629829|2021-10-08T18:09:53Z|         موفق|       1400|          07|        16|
|379627345|2021-10-08T18:09:15Z|          الی|       1400|          0

-------------------------------------------
Batch: 102
-------------------------------------------
+---------+--------------------+----------+-----------+------------+----------+
|       id|            sendTime|senderName|persianYear|persianMonth|persianDay|
+---------+--------------------+----------+-----------+------------+----------+
|379630080|2021-10-08T18:09:54Z|      موفق|       1400|          07|        16|
|379629829|2021-10-08T18:09:53Z|      موفق|       1400|          07|        16|
|379627345|2021-10-08T18:09:15Z|       الی|       1400|          07|        16|
+---------+--------------------+----------+-----------+------------+----------+

-------------------------------------------
Batch: 103
-------------------------------------------
+---------+--------------------+-------------+-----------+------------+----------+
|       id|            sendTime|   senderName|persianYear|persianMonth|persianDay|
+---------+--------------------+-------------+-----------+------------+----

-------------------------------------------
Batch: 110
-------------------------------------------
+---------+--------------------+----------+-----------+------------+----------+
|       id|            sendTime|senderName|persianYear|persianMonth|persianDay|
+---------+--------------------+----------+-----------+------------+----------+
|379630080|2021-10-08T18:09:54Z|      موفق|       1400|          07|        16|
+---------+--------------------+----------+-----------+------------+----------+

-------------------------------------------
Batch: 111
-------------------------------------------
+---------+--------------------+-------------+-----------+------------+----------+
|       id|            sendTime|   senderName|persianYear|persianMonth|persianDay|
+---------+--------------------+-------------+-----------+------------+----------+
|379629829|2021-10-08T18:09:53Z|         موفق|       1400|          07|        16|
|379627345|2021-10-08T18:09:15Z|          الی|       1400|          0

-------------------------------------------
Batch: 112
-------------------------------------------
+---------+--------------------+----------+-----------+------------+----------+
|       id|            sendTime|senderName|persianYear|persianMonth|persianDay|
+---------+--------------------+----------+-----------+------------+----------+
|379630080|2021-10-08T18:09:54Z|      موفق|       1400|          07|        16|
|379629829|2021-10-08T18:09:53Z|      موفق|       1400|          07|        16|
+---------+--------------------+----------+-----------+------------+----------+

-------------------------------------------
Batch: 113
-------------------------------------------
+---------+--------------------+-------------+-----------+------------+----------+
|       id|            sendTime|   senderName|persianYear|persianMonth|persianDay|
+---------+--------------------+-------------+-----------+------------+----------+
|379627345|2021-10-08T18:09:15Z|          الی|       1400|          07| 

-------------------------------------------
Batch: 118
-------------------------------------------
+---------+--------------------+----------+-----------+------------+----------+
|       id|            sendTime|senderName|persianYear|persianMonth|persianDay|
+---------+--------------------+----------+-----------+------------+----------+
|379630080|2021-10-08T18:09:54Z|      موفق|       1400|          07|        16|
+---------+--------------------+----------+-----------+------------+----------+

-------------------------------------------
Batch: 119
-------------------------------------------
+---------+--------------------+-------------+-----------+------------+----------+
|       id|            sendTime|   senderName|persianYear|persianMonth|persianDay|
+---------+--------------------+-------------+-----------+------------+----------+
|379629829|2021-10-08T18:09:53Z|         موفق|       1400|          07|        16|
|379627345|2021-10-08T18:09:15Z|          الی|       1400|          0

### Submit Sample Spark App in Pyspark Container Bash 

- Go to Pyspark Shell :
```bash
docker exec -it pyspark bash
```
- cd /opt/spark-app/
- run this command :
```bash
unset PYSPARK_DRIVER_PYTHON
spark-submit --master  spark-master:7077  --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2 prepare_df.py
export PYSPARK_DRIVER_PYTHON=python
```